In [1]:
import numpy as np
from gurobipy import *
import random
import matplotlib.pyplot as plt
import csv

In [2]:
data=[]

data = np.loadtxt('C:/Users/user/test/data/breast-cancer-wisconsin.data',delimiter=',',dtype=str)

In [3]:
data=data[:,1:]

L=len(data)
l=len(data[0])-1
cn = len(np.unique(list(zip(*data))[l]))

x=[]
for i in range(L):
    x.append(i)
    for j in range(l):
        if data[i,j]=='?':
            x.remove(i)
data = data[x]
data = data.astype('float64')

In [4]:
L=len(data)
l = len(data[0])-1
data1 = np.c_[data,np.zeros(L)]

np.random.shuffle(data1)

train=data1[:int((len(data1)+1)*.80)]
test=data1[int(len(data1)*.80+1):]

In [5]:
l

9

In [6]:

ci =2 # class identifier

smallnum=1.e-3
bignum=1.e+3


In [7]:

ltr=len(train)


Atrain=list()
Btrain=list()
apl=list()
bpl=list()
Cl=list()

for i in range(ltr):
    if train[i, -2] == ci:
        Atrain.append(train[i])
    else :
        Btrain.append(train[i])

Atrain = np.array(Atrain)
Btrain = np.array(Btrain)


pl=[]    # p를ㄹ 순차적으로 집어넣어줌
rsl = []
itera = 1
oldfuzzy = []


while itera > 0:
    p = [0]*l
    rs = random.random()
    random.seed(rs)
    rsl.append(rs) # random seed 저장해서 같은 값나오나 비교
    
    # 무한루프로 랜덤값 계속 찾기
    while 0 == 0:
        for i in range(l):
            p[i] += random.uniform(-1.0, 1.0)
        if sum(p[i]*p[i] for i in range(l)) >= 0.5:
            pl.append(p)
            break
            
    lA = len(Atrain)
    lB = len(Btrain)
    
    # 논문 거기 반복
    while 1 == 1:
        m = Model()
        ap = m.addVar(vtype=GRB.CONTINUOUS, name='ap', ub=bignum, lb=-bignum)
        bp = m.addVar(vtype=GRB.CONTINUOUS, name='bp', ub=bignum, lb=-bignum)
        
        m.setObjective(ap - bp, GRB.MAXIMIZE)
        
        C = []
        for i in range(l):
            C.append(m.addVar(vtype=GRB.CONTINUOUS, name = "C[{}]".format(i), ub=bignum, lb=-bignum))
            
        for i in range(lA):
            if Atrain[i, -1] == 0:
                m.addConstr(sum(C[j]*Atrain[i,j] for j in range(l)) - ap - smallnum >= 0, name = "ctp[{}]".format(i))
                    
        for i in range(lB):
            if Btrain[i, -1] == 0:
                m.addConstr(sum(C[j]*Btrain[i,j] for j in range(l)) - bp + smallnum <= 0, name = "ctq[{}]".format(i))
                    
        m.addConstr(sum(p[i]*C[i] for i in range(l)) >= .5*(.5+sum(p[i]*p[i] for i in range(l))), name = "null")
            
        m.update()
        m.optimize()
            
            # C 수렴하면 break
        k = 0
        for i in range(l):
            if (abs(C[i].x - p[i]) < smallnum):
                k += 1
                    
        if k == l:
            break
                
            # 수렴 안 하면
        for i in range(l):
            p[i] = C[i].x
        
    CI= 0
    #del Cl[:]
    
#####################################################???????????????????????????    
    
    for i in range(l):
        if itera >1 and abs(Cl[-1][i] - C[i].x)>smallnum:
            CI+=1
        
    Cll = []
        
    if itera == 1:
        for i in range(l):
            Cll.append(C[i].x)
                
        Cl.append(Cll)
        apl.append(ap.x)
        bpl.append(bp.x)
            
        sA, sB = 0, 0
            
        for i in range(lA):
            if (Atrain[i, -1] == 0)  and (sum(C[j].x * Atrain[i, j] for j in range(l)) - bp.x > 0):
                Atrain[i, -1] =1
                sA +=1
        print("Number of classifeid A points is : ", sA)

        for i in range(lB):
            if (Btrain[i, -1] == 0)  and (sum(C[j].x * Btrain[i, j] for j in range(l)) - ap.x > 0):
                Btrain[i, -1] =1
                sB +=1
        print("Number of classifeid B points is : ", sB)
                    
        #print(Atrain)
        #print(Btrain)

            
        #섞여있는 점 개수 파악
            
        fuzzy = 0
        for i in range(lA):
            if Atrain[i, -1] == 0:
                fuzzy += 1
                    
        for i in range(lB):
            if Btrain[i, -1] == 0:
                fuzzy += 1
                    
        if ap.x >= bp.x or fuzzy == 0:
            itera = 0
            print('-----')
            print('Successful termination with 0 misclassification!')
        else:
            oldfuzzy.append(fuzzy)
            itera += 1
                
    elif itera>1:
            
        if CI ==l and abs(apl[-1] -ap.x) > smallnum and abs(bpl[-1] -bp.x) >smallnum:
                                     
            for i in range(l):
                Cll.append(C[i].x)
                
            Cl.append(Cll)
            print('2번째 l ')
            print(l)
            print(Cl)
            apl.append(ap.x)
            bpl.append(bp.x)
                
            sA,sB = 0,0
        
            for i in range(lA):
                if(Atrain[i][-1]==0) and (sum(C[j].x*Atrain[i,j] for j in range(l)) -bp.x >0):
                    Atrain[i][-1]=1
                    sA+=1
            print('Number of classified A points is  ', sA)
                
            for i in range(lB):
                if(Btrain[i][-1]==0) and (sum(C[j].x*Btrain[i,j] for j in range(l)) -ap.x >0):
                    Btrain[i][-1]=1
                    sB+=1
            print('Number of classified B points is ', sB)
            #print(Atrain)
            #print(Btrain)
            
            
            fuzzy = 0
            for i in range(lA):
                if Atrain[i, -1] == 0:
                    fuzzy += 1
                    
            for i in range(lB):
                if Btrain[i, -1] == 0:
                    fuzzy += 1
                    
            if ap.x >= bp.x or fuzzy == 0:
                itera = 0
                print('-----')
                print('Successful termination with 0 misclassification!')
            elif itera >3 and oldfuzzy[-2] == oldfuzzy[-1] ==fuzzy :
                itera =0
                print('------')
                print('Fail to classify : termination with ' , fuzzy, ' misclassifications')
                print('------')
            else :
                oldfuzzy.append(fuzzy)
                itera +=1

9
Academic license - for non-commercial use only
Optimize a model with 548 rows, 11 columns and 5479 nonzeros
Coefficient statistics:
  Matrix range     [8e-02, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+03, 1e+03]
  RHS range        [1e-03, 1e+00]
Presolve removed 0 rows and 186 columns
Presolve time: 0.03s
Presolved: 11 rows, 373 columns, 3630 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+03   0.000000e+00   2.415000e+06      0s
      87   -7.4118909e+00   0.000000e+00   0.000000e+00      0s

Solved in 87 iterations and 0.07 seconds
Optimal objective -7.411890947e+00
Optimize a model with 548 rows, 11 columns and 5479 nonzeros
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+03, 1e+03]
  RHS range        [1e-03, 3e+00]
Presolve removed 0 rows and 186 columns
Presolve time: 0.01s
Presolved: 11 rows, 373 columns, 3630 nonzeros

Iteration    Objective

In [8]:
l

9

In [9]:
Atest=[]
Btest=[]

Cl = np.array(Cl)
fail=0
ltst = len(test)

for i in range(ltst):
    if test[i,-2]==ci:
        Atest.append(test[i])
    else :
        Btest.append(test[i])

Atest = np.array(Atest)
Btest=np.array(Btest)

lA=len(Atest)
lB=len(Btest)
c=len(Cl)

ptest = np.zeros((c,lA))
qtest = np.zeros((c,lB))



In [10]:
for i in range(c):
    for j in range(lA):
        for k in range(l):
            ptest[i,j] += Cl[i,k]*Atest[j,k]

for i in range(c):
    for j in range(lB):
        for k in range(l):
            qtest[i,j] += Cl[i,k]*Btest[j,k]
            
w=0

while w<c:
    lA=len(Atest)
    lB=len(Btest)
    
    rmpostest=[]
    rmnegtest=[]
    
    for i in range(lA):
        if(ptest[w,i]-apl[w]<0):
            fail += 1
    for i in range(lB):
        if(qtest[w,i]-bpl[w]>0):
            fail+=1
    s=0
    
    for i in range(lA):
        if(ptest[w,i]-bpl[w]<0):
            rmpostest.append(i)
            
            if s==0:
                s=1
    for i in range(lB):
        if(qtest[w,i]-apl[w]>0):
            rmnegtest.append(i)
            
            if s==0:
                s=1
    Atest = Atest[rmpostest]
    Btest =Btest[rmnegtest]
    
    w+=1
success_rate = (ltst-fail)*100/ltst
print(success_rate)
            

82.3529411764706
